<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import biom

from adjustText import adjust_text
import seaborn as sns
from scipy.stats import pearsonr
import warnings
warnings.simplefilter("ignore")

import matplotlib
import matplotlib.pyplot as plt
#import matplotlib as mpl
#mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

cyto_dir = '../sfari/data/cytokines'

amp_dir = '../sfari/data/sra'

Clean up Cytokine metadata

In [2]:
cyto_md = pd.read_excel(f'{cyto_dir}/Raw cytokine data.xlsx', index_col=0, skiprows=1)
cyto_md = cyto_md[['IL-6.1', 'TGFb.1', 'IL-1b.1', 'IL-4.1', 'IFN-γ.1']]

amp_md = pd.read_table(f'{amp_dir}/Zurita2019/sample_metadata.txt', index_col=0)
amp_md['host_subject_id'] = list(map(lambda x: int(x.split('.')[0][1:]), amp_md.index))

def replace_f(x, fill):
    if x == '.':
        return fill
    else:
        return float(x)

cyto_md['IL-6.1'] = cyto_md['IL-6.1'].apply(lambda x: replace_f(x, 10.))
cyto_md['IL-4.1'] = cyto_md['IL-4.1'].apply(lambda x: replace_f(x, 10.))
cyto_md['IFN-γ.1'] = cyto_md['IFN-γ.1'].apply(lambda x: replace_f(x, 4.69))

def _standardize(x):
    x = x.astype(np.float32)
    return (x - x.min()) / (x.max() - x.min())
cyto_md = cyto_md.apply(_standardize)


amp_md = pd.merge(amp_md, cyto_md, left_on='host_subject_id', right_index=True)
amp_md.columns = list((map(lambda x: x.replace('.1', ''), amp_md.columns)))
amp_md.columns = list((map(lambda x: x.replace(u'γ', 'gamma'), amp_md.columns)))
amp_md.columns = list((map(lambda x: x.replace('-', ''), amp_md.columns)))

# separate training labels
def train_f(x):
    if np.random.rand() < 0.9:
        return 'Train'
    else:
        return 'Test'
    
amp_md['Train'] = list(map(train_f, amp_md.index))

amp_md.to_csv(f'{amp_dir}/Zurita2019/cytokine_metadata.txt', sep='\t')

Load cytokine differentials from Songbird

In [3]:
diffs = []
for i in range(1, 13):
    fname = f'{amp_dir}/Zurita2019/cytokine-differentials-{i}/differentials.tsv'
    df = pd.read_table(fname)
    df['sample'] = i
    diffs.append(df)
diffs = pd.concat(diffs)


table = biom.load_table('../sfari/data/sra/Combined/age_sex_matched-v2.biom')
table.filter(amp_md.index)
table.remove_empty()

counts = table.to_dataframe().T
counts = counts.loc[amp_md.index]
cols = ['IL6', 'TGFb', 'IL1b', 'IL4']
avg_diffs = diffs[['featureid'] + cols + ['sample']].groupby('featureid').mean()
avg_diffs = avg_diffs[cols]
avg_diffs.index = list(map(str, avg_diffs.index))

Perform correlations with balances

In [4]:
def top_balance(cts, vec, k):
    svec = vec.sort_values()
    lo = svec.head(k).index
    hi = svec.tail(k).index
    num = np.log(cts.loc[:, hi] + 1).mean(axis=1)
    denom = np.log(cts.loc[:, lo] + 1).mean(axis=1)
    balance = num - denom
    return balance

def microbe_balance(cts, taxa_md):
    """ \log(\frac{Enterococcus + Bifidobacterium}{Prevotella + Desulfovibrio}) """
    ent = np.log(cts.loc[:, taxa_md['genus'] == 'Enterococcus'] + 1).mean(axis=1)
    bif = np.log(cts.loc[:, taxa_md['genus'] == 'Bifidobacterium'] + 1).mean(axis=1)
    prv = np.log(cts.loc[:, taxa_md['genus'] == 'Prevotella'] + 1).mean(axis=1)
    dsv = np.log(cts.loc[:, taxa_md['genus'] == 'Desulfovibrio'] + 1).mean(axis=1)
    num = ent + bif
    denom = prv + dsv
    balance = num - denom
    return balance

plt.rcParams['text.usetex'] = True

k = 50
fig, ax = plt.subplots(1, len(cols), figsize=(10, 3))
for i, cyto in enumerate(cols):
    il = top_balance(counts, avg_diffs[cyto], k)
    print(cyto, pearsonr(amp_md[cyto], il))
    ax[i].scatter(amp_md[cyto], il)
    ax[i].set_xlabel(cyto)
    ax[i].set_ylabel(f'log(high {cyto} / low {cyto})')
#ax[0].set_xlabel('IL-1$\beta$')
#ax[0].set_xlabel('IL-1')
plt.tight_layout()

IL6 (0.7309489444600247, 5.744446700604622e-08)
TGFb (0.6154535960864651, 1.8480087950185067e-05)
IL1b (0.33569322627441994, 0.0318960728081295)
IL4 (0.3266710599998554, 0.03710551317302614)


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode

(/dev/shm/.cache-jmorton/matplotlib/tex.cache/1acea6f6c115d0ec7a634ed0529287b9.
tex
LaTeX2e <2017-04-15>
Babel <3.17> and hyphenation patterns for 3 language(s) loaded.

! LaTeX Error: File `article.cls' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: cls)

Enter file name: 
! Emergency stop.
<read *> 
         
l.3 \newcommand
               {\mathdefault}[1]{#1}^^M
No pages of output.
Transcript written on 1acea6f6c115d0ec7a634ed0529287b9.log.




Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x1554198af820> (for post_execute):


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode

(/dev/shm/.cache-jmorton/matplotlib/tex.cache/1acea6f6c115d0ec7a634ed0529287b9.
tex
LaTeX2e <2017-04-15>
Babel <3.17> and hyphenation patterns for 3 language(s) loaded.

! LaTeX Error: File `article.cls' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: cls)

Enter file name: 
! Emergency stop.
<read *> 
         
l.3 \newcommand
               {\mathdefault}[1]{#1}^^M
No pages of output.
Transcript written on 1acea6f6c115d0ec7a634ed0529287b9.log.




RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2018) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode

(/dev/shm/.cache-jmorton/matplotlib/tex.cache/1acea6f6c115d0ec7a634ed0529287b9.
tex
LaTeX2e <2017-04-15>
Babel <3.17> and hyphenation patterns for 3 language(s) loaded.

! LaTeX Error: File `article.cls' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: cls)

Enter file name: 
! Emergency stop.
<read *> 
         
l.3 \newcommand
               {\mathdefault}[1]{#1}^^M
No pages of output.
Transcript written on 1acea6f6c115d0ec7a634ed0529287b9.log.




<Figure size 720x216 with 4 Axes>

In [ ]:
import os
os.environ["PATH"] += os.pathsep +'/usr/bin/latex'

Load Zurita differentials

In [ ]:
from util import extract_differentials, ranking
from plot import (rankplot, networkplot, vectorplot)

# load differentials
posterior_name = 'age_sex_matched_posterior'
#amp_fname = f'../sfari/data/sra/Combined/age_sex_matched_posterior/amp_differentials-v5.nc'
amp_fname = f'../sfari/data/sra/Zurita2019/age_sex_matched_posterior'
amp_diffs = extract_differentials(amp_fname)

# Compute statistical tests for each data layer
# Here, we will only focus on the top 10% of the features
amp_stats = ranking(amp_diffs, reference_percentile=50, log_probs=True)

combined_amp_stats = pd.read_csv('../results/amp_combined_diffs.csv', index_col=0)

#amp_stats = pd.merge(amp_stats, taxonomy, left_index=True, right_index=True, how='left')
#amp_stats = amp_stats.set_index('GOTU')
taxa_cols = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
amp_stats = pd.merge(amp_stats, combined_amp_stats[['group', 'Zurita2019'] + taxa_cols], 
                     left_index=True, right_index=True, how='outer')

# create a consensus, focusing on microbes that agree between Berding and the combined analysis
def consensus_f(x):
    if x['Zurita2019'] == x['group']:
        return x['group']
    return 'neutral'
amp_stats['Zurita_group_consensus'] = amp_stats.apply(consensus_f, axis=1)

Only focus on a few genera

In [ ]:
bugs = ['Enterococcus', 'Bifidobacterium', 
        'Prevotella', 'Desulfovibrio', 'Bacteroides']


def genus_f(x):
    if pd.isnull(x):
        return 'Other'
    #if x.split('_')[0] == 'Enterococcus':
    #    return 'Enterococcus'
    if x.split('_')[0] == 'Desulfovibrio':
        return 'Desulfovibrio'
    if x.split('_')[0] == 'Bacteroides':
        return 'Bacteroides'
    
    if x in bugs:
        return x
    else:
        return 'Other'

amp_stats['genera'] = amp_stats['genus'].apply(genus_f)
#amp_stats = pd.merge(amp_stats, taxa_md, left_index=True, right_index=True)


Focus on IL6 and TGFb differentials

In [ ]:
df = diffs[['featureid', 'IL6', 'sample']].pivot(index='sample', columns='featureid', values='IL6')
il6_stats = ranking(df.T, reference_percentile=50, log_probs=True)[['mean']].rename(columns={'mean': 'IL6'})
cyto_stats = pd.merge(il6_stats, amp_stats, left_index=True, right_index=True, how='outer')
cyto_stats = cyto_stats.dropna(subset=['IL6', 'mean'])

df = diffs[['featureid', 'TGFb', 'sample']].pivot(index='sample', columns='featureid', values='TGFb')
tgf_stats = ranking(df.T, reference_percentile=50, log_probs=True)[['mean']].rename(columns={'mean': 'TGFb'})
cyto_stats = pd.merge(tgf_stats, cyto_stats, left_index=True, right_index=True, how='outer')
cyto_stats = cyto_stats.dropna(subset=['TGFb', 'mean'])

In [ ]:
plt.scatter(cyto_stats['IL6'], cyto_stats['mean'])
r, p = pearsonr(cyto_stats['IL6'], cyto_stats['mean'])
print('r=', r, 'p=', p)

In [ ]:
plt.scatter(cyto_stats['TGFb'], cyto_stats['mean'])
r, p = pearsonr(cyto_stats['TGFb'], cyto_stats['mean'])
print('r=', r, 'p=', p)

Final figure

In [ ]:
plt.rcParams['text.usetex'] = True
sns.set_style('white')

fs = 22
ls = 20

# rename for latex
cytokine_k = ['IL6', 'TGFb', 'IL1b', 'IL4', 'IFNgamma']
#cytokine_v = ['IL6', r'TGF-$\beta$', r'IL-1$\beta$', 'IL4', r'IFN-$\gamma$']
cytokine_v = [r'IL\textrm{-}6', r'TGF\textrm{-}\beta', r'IL-1$\beta$', 'IL4', r'IFN-$\gamma$']
lookup = dict(zip(cytokine_k, cytokine_v))

cytokine_names = cytokine_k

fig, ax = plt.subplots(2, 2, figsize=(12, 9), sharey=False)

order = ['Other', 'Bifidobacterium', 'Bacteroides', 'Prevotella', 'Desulfovibrio']
bugs = ['Prevotella', 'Bacteroides', 'Bifidobacterium', 
        'Desulfovibrio']
palette = dict(zip(bugs, list(sns.color_palette())[:len(bugs)]))
palette['Other'] = '#E0E0E0'
palette['Prevotella'] = '#7245ff'
palette['Desulfovibrio'] = 'k'
species_abv = {'Prevotella copri': '$P. copri$',
               'Bifidobacterium faecale': '$B. faecale$',
               'Bacteroides_E fragilis': '$B. fragilis$'}

## Scatter plots against cytokine concentrations
j = 1
eps = 1e-6
for i, cyto in enumerate(['IL6', 'TGFb']):
    il = top_balance(counts, avg_diffs[cyto], k)
    #il = microbe_balance(counts, taxmd)
    ax[j][i].scatter(amp_md[cyto], il, color='k', s=10)
    print(cyto, pearsonr(amp_md[cyto], il))
    #ax[j][i].set_ylabel(f'log(high {cyto} / low {cyto})')
    ax[j][i].set_ylabel('')
    ax[j][i].set_xlabel('$' + cytokine_v[i] + '$ (conc.)', fontsize=fs)
    ax[j][i].set_ylim([-3, 3])
    #ax[j][i].set_xlim([-0.05, 1.05])
    ax[j][i].tick_params(axis='x', labelsize=ls)
    ax[j][i].tick_params(axis='y', labelsize=ls)

ylabel = r'$\log(\frac{\textrm{Top} \;' + str(k) + '\; \\textrm{microbes}}{\\textrm{Bottom} \;' + str(k) + '\; \\textrm{microbes}})$'
ax[j][0].set_ylabel(ylabel, fontsize=fs, labelpad=100, rotation=0)

all_texts = []
j = 0
for i, cyto in enumerate(cytokine_names[:2]):
    df = diffs[['featureid', cyto, 'sample']].pivot(index='sample', columns='featureid', values=cyto)
    df_stats = ranking(df.T, reference_percentile=50, log_probs=True)
    # realign
    #df_stats['genome'] = df_stats['genome'].astype(np.str)
    df_ = df_stats[['mean']]
    df_ = df_.rename(columns={'mean': 'cytokine_mean'})
    df_stats = pd.merge(df_, amp_stats, left_index=True, right_index=True)
    df_stats = df_stats.rename(columns={'mean': 'amp_mean'})

    df_stats.index = list(map(str, df_stats.index))    
    legend = (i == (len(cytokine_names) - 2))
    #ax[j][i].vlines(lo, -1.2, 1.2, 'r', linestyle=':')
    #ax[j][i].vlines(hi, -1.2, 1.2, 'r', linestyle=':')

    #df_stats['ordering'] = (df_stats['genera'] != 'Other')
    df_stats['ordering'] = df_stats['genera'].apply(
        lambda x: {'Desulfovibrio': 1,  'Bifidobacterium': 3, 'Prevotella':3, 'Bacteroides': 4, 'Other': 5}[x])
    df_stats = df_stats.sort_values('ordering', ascending=False)

    sns.scatterplot(data=df_stats, x='cytokine_mean', y='amp_mean', hue='genera', 
                    ax=ax[j][i], legend=legend,  zorder=10,
                    s=50, hue_order=order, palette=palette, alpha=1)
    xlabel = r'$\log(\frac{\textrm{high }' + lookup[cyto] + '}{ \\textrm{low }' +  lookup[cyto] + '})$'

    ax[j][i].set_xlabel(xlabel, fontsize=fs)
    ax[j][i].set_ylabel("")
    ax[j][i].set_xlim([-11, 11])
    #ax[j][i].set_ylim([-5, 5])
    ax[j][i].tick_params(axis='x', labelsize=ls)
    ax[j][i].tick_params(axis='y', labelsize=ls)

    idx = [x in species_abv.keys() for x in cyto_stats['species']]
    subcyto = cyto_stats.iloc[idx]
    x = subcyto[cyto].values
    y = subcyto['mean'].values
    s = subcyto['species'].values

    dx, dy = 0, 0
    if cyto == 'TGFb': 
        dx, dy = 0, 0
    if cyto == 'IL6': 
        dx, dy = 0, 0

    #texts = [ax[j][i].text(x[k] + dx, y[k] + dy, f"\\textit{{{species_abv[s[k]]}}}", fontsize=16,
    #                    ha='center', va='center', zorder=20, color='k') 
    #         for k in range(len(x))]
    #adjust_text(texts, ax=ax[j][i])

    lo = df_stats['cytokine_mean'].sort_values().iloc[k]
    hi = df_stats['cytokine_mean'].sort_values().iloc[-k]
    ax[j][i].axvspan(hi, 14, alpha=0.5, color='lavender', zorder=0)
    ax[j][i].axvspan(-14, lo, alpha=0.5, color='lightsalmon', zorder=0)
ax[j][0].set_ylim([-6, 6])
ax[j][1].set_ylim([-6, 6])



ax[j][0].set_ylabel(r'$\log(\frac{\textrm{ASD}}{\textrm{Control}}) + K$', 
                    fontsize=fs, rotation=0, labelpad=80)
plt.tight_layout(w_pad=0.5)

In [ ]:
cyto_stats.query('genera == "Desulfovibrio"')

In [ ]:
cyto_stats

In [ ]:
cyto_stats.to_csv('../results/supplemental_tables/Table_S9.csv')

In [ ]:
print_cols = ['IL6', 'TGFb', '5%', 'mean', '95%', 'pvalue', 'group', 'Zurita_group_consensus', 'species']

In [ ]:
cyto_stats['IL6'].quantile([0.05, 0.25, 0.5, 0.75, 0.95])

In [ ]:
(cyto_stats
 .query("Zurita_group_consensus == 'num'")
 .sort_values('IL6')[print_cols]
 .dropna()
 .query("`5%` > 0"))

In [ ]:
(cyto_stats
 .sort_values('IL6')[print_cols]
 .dropna()
 .query("`IL6` < -1.21")).head(50)

In [ ]:
(cyto_stats
 .sort_values('IL6')[print_cols]
 .dropna()
 .query("`IL6` > 1.52")).tail(50)

In [ ]:
cyto_stats['TGFb'].quantile([0.05, 0.25, 0.5, 0.75, 0.95])

In [ ]:
(cyto_stats
 .sort_values('TGFb')[print_cols]
 .dropna()
 .query("`5%` > 0"))

In [ ]:
(cyto_stats
 .sort_values('TGFb')[print_cols]
 .dropna()
 .query(f"`TGFb` < -1.07")).head(50)

In [ ]:
(cyto_stats
 .sort_values('TGFb')[print_cols]
 .dropna()
 .query(f"`TGFb` > 1.12"))

In [ ]:
(cyto_stats
 .query("Zurita_group_consensus == 'denom'")
 .sort_values('TGFb')[print_cols]
 .dropna()
 .query("`95%` < 0"))

In [ ]:
cyto_stats.query("genera == 'Desulfovibrio'")[print_cols]

In [ ]:
cyto_stats.query("genera == 'Enterococcus'")[print_cols]

In [ ]:
cyto_stats.query("genus == 'Prevotella'")[print_cols]

In [ ]:
cyto_stats.query("genera == 'Bifidobacterium'")[print_cols]

In [ ]:
cyto_stats.query("genera == 'Bacteroides'")[print_cols]

In [ ]:
cyto_stats.query("group == 'num'").sort_values('TGFb').tail(50)

In [ ]:
cyto_stats.query("group == 'denom'").sort_values('TGFb').head(50)

In [ ]:
amp_stats.loc['AY305310']

In [ ]:
amp_stats.loc['G010592515']

In [ ]:
amp_stats.loc['JX861483']

In [ ]:
amp_stats.loc['G900113595']

In [ ]:
cyto_stats

In [ ]:
diffs